References:
- https://docs.python.org/3/reference/datamodel.html
- Fluent Python by Luciano Ramalho. Chapter 14: Iterables, Iterators, and Generators

# Sentence Take #1: A Sequence of Words

In [1]:
import re
import reprlib

Extracts words from a text by index:

In [2]:
RE_WORD = re.compile(r'\w+')

In [3]:
class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)  # returns a list with all nonoverlapping matches of re

    def __getitem__(self, index):
        return self.words[index]  # holds  the result of .findall

    def __len__(self):  # to complete the sequence protocol
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)  # a utility function to generate abbreviated string repr

In [4]:
s = Sentence('Beautiful is better than ugly.')
s

Sentence('Beautiful is...er than ugly.')

In [5]:
for word in s:
    print(word)

Beautiful
is
better
than
ugly


As an iterable, the Sentence object can be used to build  lists and other iterable types:

In [6]:
list(s)

['Beautiful', 'is', 'better', 'than', 'ugly']

## `__iter__`

In [7]:
class Foo:
    def __iter__(self):
        pass

In [8]:
from collections import abc
issubclass(Foo, abc.Iterable)

True

In [9]:
f = Foo()
isinstance(f, abc.Iterable)

True

In [10]:
issubclass(Sentence, abc.Iterable)

False

In [11]:
isinstance(s, abc.Iterable)

False

# Iterables Versus Iterators

In [12]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [13]:
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it  # release reference to it
        break

A
B
C


<img src="iter.png" width="50%">

In [14]:
s3 = Sentence('Explicit is better than implicit.')
it = iter(s3)
it

In [15]:
next(it)

'Explicit'

In [16]:
next(it)

'is'

In [17]:
list(it)

['better', 'than', 'implicit']

In [18]:
list(iter(s3))

['Explicit', 'is', 'better', 'than', 'implicit']

# Sentence Take #2: A Classic Iterator

In [19]:
class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):  # add the iter method
        return SentenceIterator(self.words)  # fulfil the iterable protocol

In [20]:
class SentenceIterator:

    def __init__(self, words):
        self.words = words  # holds a reference to a list of words
        self.index = 0  # used to  determine the next word to fetch

    def __next__(self):
        try:
            word = self.words[self.index]  # get the word at index
        except IndexError:
            raise StopIteration()  
        self.index += 1 
        return word  

    def __iter__(self):  
        return self

In [21]:
issubclass(Sentence, abc.Iterable)

True

In [22]:
issubclass(Sentence, abc.Iterator)

False

In [23]:
issubclass(SentenceIterator, abc.Iterator)

True

# Sentence Take #3: A Generator Function

In [24]:
class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:  
            yield word  # yield the current word
        return # not needed

## How a Generator Function Works:

In [25]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [26]:
gen_123

<function __main__.gen_123()>

In [27]:
gen_123()

<generator object gen_123 at 0x1107d46d0>

In [28]:
g = gen_123()
next(g)

1

In [140]:
next(g)

StopIteration: 

In [30]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end')

In [31]:
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end


# Sentence Take #4: A Lazy Implementation

In [32]:
class Sentence:

    def __init__(self, text):
        self.text = text  # no need for a word list

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # build an iterator over the matches, yielding MatchObject Instances
            yield match.group()  # extracts the actual matched text

# Sentence Take #5: A Generator Expression

In [33]:
# list comprehension
res1 = [x*3 for x in gen_AB()]

start
continue
end


In [34]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [35]:
# generator expression
res2 = (x*3 for x in gen_AB())

In [36]:
res2

<generator object <genexpr> at 0x1107f47d0>

In [37]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end


In [38]:
class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

# Arithmetic Progression Generator

In [39]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end  # None -> "infinite" series

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        while forever or result < self.end:
            yield result
            result += self.step

In [40]:
ap = ArithmeticProgression(0, 1, 3)

In [41]:
list(ap)

[0, 1, 2]

In [42]:
ap = ArithmeticProgression(1, .5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [43]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [44]:
from fractions import Fraction

In [45]:
ap = ArithmeticProgression(0, Fraction(1, 3), 1)
list(ap)

[Fraction(0, 1), Fraction(1, 3), Fraction(2, 3)]

In [46]:
from decimal import Decimal

In [47]:
ap = ArithmeticProgression(0, Decimal('.1'), .3)
list(ap)

[Decimal('0'), Decimal('0.1'), Decimal('0.2')]

In [48]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

## Arithmetic Progression  with itertools

In [49]:
import itertools

In [50]:
gen = itertools.count(1, .5)
next(gen)

1

In [51]:
next(gen)

1.5

### `itertools.takewhile`
produces a generator that consumes another generator and stops when a  given predicate eveluates to False.

In [52]:
gen = itertools.takewhile(lambda n: n<3, itertools.count(1, .5))

In [53]:
list(gen)

[1, 1.5, 2.0, 2.5]

In [54]:
def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

#  Generator Functions in  the Standard Library
<img src="filter.png" width="50%">

In [55]:
def vowel(c):
    return c.lower() in 'aeiou'

In [56]:
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [57]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [58]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [59]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [60]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [61]:
list(itertools.islice('Aardvark', 4)) # [:4]

['A', 'a', 'r', 'd']

In [62]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [63]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

<img src="map.png" width="50%">

In [64]:
sample = [9, 8, 7, 6, 5, 4, 3, 2, 1] 

In [65]:
list(itertools.accumulate(sample))

[9, 17, 24, 30, 35, 39, 42, 44, 45]

In [66]:
list(itertools.accumulate(sample, min))

[9, 8, 7, 6, 5, 4, 3, 2, 1]

In [67]:
sample = [9, 8, 7, 6, 5, 1, 4, 3, 2, 1] 
list(itertools.accumulate(sample, min))

[9, 8, 7, 6, 5, 1, 1, 1, 1, 1]

In [68]:
list(itertools.accumulate(sample, max))

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [69]:
sample = [1, 2, 3, 9, 8, 7, 6, 5, 1, 4, 3, 2, 1] 
list(itertools.accumulate(sample, max))

[1, 2, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]

In [70]:
import operator

In [71]:
sample = [9, 8, 7, 6, 5, 4, 3, 2, 1] 
list(itertools.accumulate(sample, operator.mul))

[9, 72, 504, 3024, 15120, 60480, 181440, 362880, 362880]

In [72]:
list(itertools.accumulate(range(1, 11), operator.mul))

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

In [73]:
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [74]:
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [75]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [76]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [77]:
list(itertools.starmap(operator.mul, enumerate('albatroz', 1))) 

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [78]:
list(itertools.starmap(lambda  a, b: b/a, enumerate(itertools.accumulate(sample), 1)))

[9.0, 8.5, 8.0, 7.5, 7.0, 6.5, 6.0, 5.5, 5.0]

<img src="genmerge.png" width="50%">

In [79]:
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [80]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [81]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [82]:
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [83]:
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [84]:
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [85]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

In [86]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [87]:
suits = 'spades hearts diamonds clubs'.split()

In [88]:
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [89]:
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [90]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [91]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [92]:
rows = itertools.product('AB', range(2), repeat=2)

In [93]:
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


<img src="expand.png" width="50%">

In [94]:
ct = itertools.count()

In [95]:
next(ct)

0

In [96]:
(next(ct), next(ct), next(ct))

(1, 2, 3)

In [97]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [98]:
cy  = itertools.cycle('ABC')

In [99]:
next(cy)

'A'

In [100]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [101]:
rp = itertools.repeat(7)

In [102]:
next(rp), next(rp)

(7, 7)

In [103]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [104]:
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [105]:
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [106]:
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [107]:
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [108]:
list(itertools.product('ABC', repeat=2)) # Cartesian  product

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [109]:
list(itertools.product('ABC', repeat=3)) # Cartesian  product

[('A', 'A', 'A'),
 ('A', 'A', 'B'),
 ('A', 'A', 'C'),
 ('A', 'B', 'A'),
 ('A', 'B', 'B'),
 ('A', 'B', 'C'),
 ('A', 'C', 'A'),
 ('A', 'C', 'B'),
 ('A', 'C', 'C'),
 ('B', 'A', 'A'),
 ('B', 'A', 'B'),
 ('B', 'A', 'C'),
 ('B', 'B', 'A'),
 ('B', 'B', 'B'),
 ('B', 'B', 'C'),
 ('B', 'C', 'A'),
 ('B', 'C', 'B'),
 ('B', 'C', 'C'),
 ('C', 'A', 'A'),
 ('C', 'A', 'B'),
 ('C', 'A', 'C'),
 ('C', 'B', 'A'),
 ('C', 'B', 'B'),
 ('C', 'B', 'C'),
 ('C', 'C', 'A'),
 ('C', 'C', 'B'),
 ('C', 'C', 'C')]

<img src="rearrange.png" width="50%">

In [110]:
list(itertools.groupby('LLLLAAAGG'))

[('L', <itertools._grouper at 0x1107eaa90>),
 ('A', <itertools._grouper at 0x1107ea2d0>),
 ('G', <itertools._grouper at 0x110812610>)]

In [111]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [112]:
for char, group in itertools.groupby('LLLLAAAGGLl'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']
L -> ['L']
l -> ['l']


In [113]:
animals = ['duck', 'cat', 'dog', 'mouse', 'goose', 'python', 'hamster']

In [114]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

4 -> ['duck']
3 -> ['cat', 'dog']
5 -> ['mouse', 'goose']
6 -> ['python']
7 -> ['hamster']


In [115]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['hamster']
6 -> ['python']
5 -> ['goose', 'mouse']
3 -> ['dog', 'cat']
4 -> ['duck']


In [116]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x11086e3c0>, <itertools._tee at 0x11086e410>]

In [117]:
g1, g2 = itertools.tee('ABC')

In [118]:
next(g1)

'A'

In [119]:
next(g2)

'A'

In [120]:
next(g2)

'B'

In [121]:
list(g1)

['B', 'C']

In [122]:
list(g2)

['C']

In [123]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

# New Syntax in Python 3.3 : `yield from`

In [124]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [125]:
s = 'ABC'
t = tuple(range(3))

In [126]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

# Iterable Reducing Functions

In [127]:
all([1, 2, 3])

True

In [128]:
all([1, 0, 3])

False

In [129]:
all([1, -2, 3])

True

In [130]:
all([])

True

In [131]:
any([1, 2, 3])

True

In [132]:
any([1, 0, 0])

True

In [133]:
any([0, 0.0])

False

In [134]:
g = (n for n in [0, 0.0, 7, 8])

In [135]:
any(g)

True

In [136]:
next(g)

8

In [137]:
l = [n for n in [0, 0.0, 7, 8]]

In [138]:
any(l)

True

In [139]:
next(l)

TypeError: 'list' object is not an iterator

In [141]:
from random import randint 

In [142]:
def d6():
    return randint(1, 6)

In [143]:
d6_iter = iter(d6, 1)

In [144]:
d6_iter

In [145]:
for roll in d6_iter:
    print(roll)